In [6]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder

class HAM10000Dataset(Dataset):
    def __init__(self, csv_file, img_dirs, transform=None, device='cuda'):
        self.data = pd.read_csv(csv_file)
        self.img_dirs = img_dirs
        self.transform = transform
        self.device = device
        
        # Encode labels
        self.label_encoder = LabelEncoder()
        self.data['encoded_label'] = self.label_encoder.fit_transform(self.data['dx'])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]['image_id'] + '.jpg'
        for img_dir in self.img_dirs:
            img_path = os.path.join(img_dir, img_name)
            if os.path.exists(img_path):
                image = Image.open(img_path).convert('RGB')
                if self.transform:
                    image = self.transform(image)
                label = self.data.iloc[idx]['encoded_label']
                return image, label
        raise FileNotFoundError(f"Image {img_name} not found in directories {self.img_dirs}")

class SLEBlock(nn.Module):
    def __init__(self, in_channels):
        super(SLEBlock, self).__init__()
        self.global_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = nn.Conv2d(in_channels, in_channels // 2, 1)
        self.fc2 = nn.Conv2d(in_channels // 2, in_channels, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, y):
        x = self.global_pool(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return y * x
        
class FASTGANGenerator(nn.Module):
    def __init__(self, latent_dim=256, ngf=64, output_size=64):
        super(FASTGANGenerator, self).__init__()
        self.output_size = output_size
        self.initial = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, ngf * 16, 4, 1, 0),
            nn.BatchNorm2d(ngf * 16),
            nn.ReLU(True)
        )
        self.layer1 = nn.Sequential(
            nn.ConvTranspose2d(ngf * 16, ngf * 8, 4, 2, 1),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True)
        )
        self.sle1 = SLEBlock(ngf * 8)
        self.layer2 = nn.Sequential(
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True)
        )
        self.sle2 = SLEBlock(ngf * 4)
        self.layer3 = nn.Sequential(
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True)
        )
        self.layer4 = nn.Sequential(
            nn.ConvTranspose2d(ngf * 2, 3, 4, 2, 1),
            nn.Tanh()
        )

    def forward(self, z):
        x = self.initial(z)
        x = self.layer1(x)
        x = self.sle1(x, x)
        x = self.layer2(x)
        x = self.sle2(x, x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x

class FASTGANDiscriminator(nn.Module):
    def __init__(self, ndf=64, input_size=64):
        super(FASTGANDiscriminator, self).__init__()
        self.input_size = input_size
        self.main = nn.Sequential(
            nn.Conv2d(3, ndf, 4, 2, 1),
            nn.LeakyReLU(0.2),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2),
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(ndf * 8, 1, 1),
            nn.Flatten(),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x)

class SelfSupervisedDiscriminator(nn.Module):
    def __init__(self, ndf=64):
        super(SelfSupervisedDiscriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, ndf, 4, 2, 1),          # 16x16 -> 8x8
            nn.LeakyReLU(0.2),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1),   # 8x8 -> 4x4
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1),  # 4x4 -> 2x2
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1),  # 2x2 -> 1x1
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2),
            nn.Conv2d(ndf * 8, ndf * 16, 1, 1, 0), # 1x1 -> 1x1
            nn.BatchNorm2d(ndf * 16),
            nn.LeakyReLU(0.2),
            nn.Conv2d(ndf * 16, 1, 1, 1, 0),      # 1x1 -> 1x1
            nn.Sigmoid()
        )
        
        # Small decoders for self-supervised learning
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(ndf * 16, ndf * 8, 4, 1, 0),   # 1x1 -> 2x2
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ndf * 8, ndf * 4, 4, 2, 1),    # 2x2 -> 4x4
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ndf * 4, ndf * 2, 4, 2, 1),    # 4x4 -> 8x8
            nn.BatchNorm2d(ndf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ndf * 2, 3, 4, 2, 1),          # 8x8 -> 16x16
            nn.Tanh()
        )

    def forward(self, x):
        features = self.main[:-1](x)            # Extract features before the final layer
        validity = self.main[-1](features)      # Compute validity score
        reconstruction = self.decoder1(features) # Reconstruct the image
        
        # Debugging Statements
        print(f"Input Image Size: {x.size()}")
        print(f"Reconstructed Image Size: {reconstruction.size()}")
        
        return validity, reconstruction

def train_step(real_imgs, generator, discriminator, g_optimizer, d_optimizer, 
               device, autoencoder_loss):
    batch_size = real_imgs.size(0)
    
    # Train Discriminator
    d_optimizer.zero_grad()
    
    real_validity = discriminator(real_imgs)
    
    z = torch.randn(batch_size, 256, 1, 1, device=device)
    fake_imgs = generator(z)
    fake_validity = discriminator(fake_imgs.detach())
    
    d_loss = (F.binary_cross_entropy(real_validity, torch.ones_like(real_validity)) +
              F.binary_cross_entropy(fake_validity, torch.zeros_like(fake_validity)))
    
    d_loss.backward()
    d_optimizer.step()
    
    # Train Generator
    g_optimizer.zero_grad()
    
    fake_validity = discriminator(fake_imgs)
    g_loss = F.binary_cross_entropy(fake_validity, torch.ones_like(fake_validity))
    
    g_loss.backward()
    g_optimizer.step()
    
    return d_loss.item(), g_loss.item(), fake_imgs

def train_fastgan(generator, discriminator, dataloader, num_epochs, device='cuda'):
    g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    autoencoder_loss = nn.MSELoss()
    
    for epoch in range(num_epochs):
        for i, (real_imgs, _) in enumerate(dataloader):
            real_imgs = real_imgs.to(device)
            
            d_loss, g_loss, fake_imgs = train_step(real_imgs, generator, discriminator, 
                                                   g_optimizer, d_optimizer, 
                                                   device, autoencoder_loss)
            
            if i % 100 == 0:
                print(f'Epoch [{epoch}/{num_epochs}], '
                      f'D_loss: {d_loss:.4f}, G_loss: {g_loss:.4f}')
                
                # Optional: Save some generated images
                if i % 500 == 0:
                    save_image(fake_imgs[:16] * 0.5 + 0.5, 
                               f'generated_images_epoch_{epoch}_batch_{i}.png', 
                               normalize=False)

class ProgressiveGrowingManager:
    def __init__(self, start_size=16, target_size=64, n_steps=3):
        self.current_size = start_size
        self.target_size = target_size
        self.n_steps = n_steps
        self.alpha = 0.0
        
    def step(self):
        self.alpha = min(1.0, self.alpha + 0.1)
        if self.alpha >= 1.0 and self.current_size < self.target_size:
            self.current_size = min(self.current_size * 2, self.target_size)
            self.alpha = 0.0
            
    def get_size(self):
        return self.current_size

class SyntheticImageClassifier:
    def __init__(self, num_classes, device='cuda'):
        self.device = device
        
        # EfficientNetV2
        self.efficientnet = models.efficientnet_v2_s(pretrained=True)
        self.efficientnet.classifier[1] = nn.Linear(self.efficientnet.classifier[1].in_features, num_classes)
        self.efficientnet = self.efficientnet.to(device)
        
        # ShuffleNetV2
        self.shufflenet = models.shufflenet_v2_x1_0(pretrained=True)
        self.shufflenet.fc = nn.Linear(self.shufflenet.fc.in_features, num_classes)
        self.shufflenet = self.shufflenet.to(device)
        
        # Transformation for input images
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
    def load_pretrained_weights(self, efficientnet_path, shufflenet_path):
        """
        Load pretrained weights for both models
        
        Args:
            efficientnet_path (str): Path to EfficientNetV2 weights
            shufflenet_path (str): Path to ShuffleNetV2 weights
        """
        self.efficientnet.load_state_dict(torch.load(efficientnet_path))
        self.shufflenet.load_state_dict(torch.load(shufflenet_path))
        
        # Set models to evaluation mode
        self.efficientnet.eval()
        self.shufflenet.eval()
    
    def classify_synthetic_images(self, synthetic_images):
        """
        Classify synthetic images using both models
        
        Args:
            synthetic_images (torch.Tensor): Tensor of synthetic images
        
        Returns:
            torch.Tensor: Mask of correctly classified images
        """
        # Resize and normalize synthetic images for classification
        resized_images = F.interpolate(synthetic_images, size=(224, 224), mode='bilinear', align_corners=False)
        normalized_images = (resized_images - resized_images.min()) / (resized_images.max() - resized_images.min())
        normalized_images = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(normalized_images)
        
        # Get predictions from both models
        with torch.no_grad():
            efficientnet_preds = self.efficientnet(normalized_images)
            shufflenet_preds = self.shufflenet(normalized_images)
        
        # Get class predictions
        efficientnet_classes = torch.argmax(efficientnet_preds, dim=1)
        shufflenet_classes = torch.argmax(shufflenet_preds, dim=1)
        
        # Create mask where both models agree
        agreed_classification_mask = (efficientnet_classes == shufflenet_classes)
        
        return agreed_classification_mask

def main():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    torch.manual_seed(42)
    np.random.seed(42)
    
    transform = transforms.Compose([
        transforms.Resize((64, 64)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    csv_file = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv'
    img_dirs = ['/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1', '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2']
    
    dataset = HAM10000Dataset(csv_file, img_dirs, transform=transform, device=device)
    
    num_classes = len(dataset.label_encoder.classes_)
    print("Unique Classes:", dataset.label_encoder.classes_)
    print("Number of Classes:", num_classes)
    
    batch_size = 64
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    
    latent_dim = 256
    generator = FASTGANGenerator(latent_dim, output_size=64).to(device)
    discriminator = FASTGANDiscriminator(input_size=64).to(device)
    
    num_epochs = 1
    
    train_fastgan(generator, discriminator, data_loader, num_epochs, device)
    
    os.makedirs('synthetic_images', exist_ok=True)
    
    # Initialize the Synthetic Image Classifier
    classifier = SyntheticImageClassifier(num_classes=num_classes, device=device)
    
    # Note: In a real scenario, you would load pretrained weights
    # classifier.load_pretrained_weights('path/to/efficientnet_weights.pth', 'path/to/shufflenet_weights.pth')
    
    synthetic_images_by_class = {}
    
    with torch.no_grad():
        for class_idx in range(num_classes):
            z = torch.randn(100, latent_dim, 1, 1).to(device)
            synthetic_images = generator(z)
            
            # Classify synthetic images
            valid_image_mask = classifier.classify_synthetic_images(synthetic_images)
            
            # Filter synthetic images based on classification
            valid_synthetic_images = synthetic_images[valid_image_mask]
            
            synthetic_images_by_class[class_idx] = valid_synthetic_images.cpu()
            
            class_name = dataset.label_encoder.inverse_transform([class_idx])[0]
            class_dir = os.path.join('synthetic_images', class_name)
            os.makedirs(class_dir, exist_ok=True)
            
            for i, img in enumerate(valid_synthetic_images):
                save_path = os.path.join(class_dir, f'synthetic_image_{i}.png')
                save_image((img * 0.5 + 0.5), save_path)
    
    print("Synthetic image generation, classification, and filtering complete!")

if __name__ == "__main__":
    main()

Using device: cuda
Unique Classes: ['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']
Number of Classes: 7
Epoch [0/1], D_loss: 1.4208, G_loss: 0.5995
Epoch [0/1], D_loss: 0.5373, G_loss: 1.5907


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/efficientnet_v2_s-dd5fe13b.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_s-dd5fe13b.pth
100%|██████████| 82.7M/82.7M [00:00<00:00, 201MB/s]
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprec

Synthetic image generation, classification, and filtering complete!
